In [1]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from utils_plotting import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import metpy
import numbers

In [2]:
data_location = 'data'
outlooks, pph, reports = read_datasets(data_location, 'labelled')

reading outlooks
reading pph
reading storm reports


In [29]:
category_dict = {
    'NONE' : -1,
    'TSTM': 0,
    'MRGL': 1,
    'SLGT': 2,
    'ENH': 3,
    'MDT': 4,
    'HIGH': 5
}


season_dict = {
    'Winter': 0,
    'Spring': 1,
    'Summer': 2,
    'Fall': 3
}

region_dict = {
    'NONE': -1,
    'West': 0,
    'Great Plains': 1,
    'Midwest': 2,
    'Northeast': 3,
    'South': 4
}

tor_dict = {
    'NONE': -1,
    'EFU': 0,
    '(E)F0': 1,
    '(E)F1': 2,
    '(E)F2': 3,
    '(E)F3': 4,
    '(E)F4': 5,
    '(E)F5': 6
}

cats = ['MAX_CAT', 'SEASON', 'REGION', 'TOR_F']
dicts = [category_dict, season_dict, region_dict, tor_dict]

In [34]:
# remove unneeded variables and x, y dimensions
data = pph.drop(['lat', 'lon', 'p_perfect_wind', 'p_perfect_sig_wind', 'p_perfect_hail', 'p_perfect_sig_hail', 'p_perfect_tor', 'p_perfect_sig_tor', 'RAMP_CAT', 'p_perfect_total', 'PPH_CAT', 'REPORT_NUM', 'WINDSP_CAT', 'HAILSZ_CAT'])


# make data numerical
def convert_strings_to_ints(data, conversion_dict):
    return np.vectorize(conversion_dict.get)(data)

for cat, dic in zip(cats, dicts):

    data[cat] = xr.apply_ufunc(
        convert_strings_to_ints,  # function to apply
        data[cat],      # input data
        kwargs={'conversion_dict': dic},  # additional arguments
        vectorize=True            # vectorize the function
    )

In [30]:
# consolidate dates 
earliest_date = max(min(outlooks['DATE']), min(pph['time']), min(reports['DATE'])) #TODO first two days of outlook dataset don't have day 3 forecast. but probably ok since not mdt
latest_date = min(max(outlooks['DATE']), max(pph['time']), max(reports['DATE']))

In [42]:
earliest_date = '200203300000'
all_pph_dates = data['time']
pph_dates = all_pph_dates[all_pph_dates <= latest_date]
pph_dates = pph_dates[pph_dates >= earliest_date]
new_data = data.sel(time = pph_dates)

In [44]:
def standardize_dataset(dataset):
    standardized_data = {}
    for var in dataset.data_vars:
        data_array = dataset[var]
        mean = data_array.mean()
        std = data_array.std()
        standardized_data[var] = (data_array - mean) / std
    return xr.Dataset(standardized_data)

new_data = standardize_dataset(new_data)

In [45]:
# do pca
new_data

<xarray.Dataset>
Dimensions:     (time: 7582)
Coordinates:
  * time        (time) object '200203300000' '200203310000' ... '202212310000'
Data variables: (12/15)
    MAX_CAT     (time) float64 0.6308 0.6308 -0.9863 ... -0.1778 -0.1778 -0.1778
    RAMP_UP     (time) float64 -0.6011 -0.6011 -0.6011 ... -0.6011 0.7821 0.7821
    RAMP_DOWN   (time) float64 0.1704 0.1704 0.1704 ... 0.1704 0.1704 0.1704
    SEASON      (time) float64 -0.4644 -0.4644 -0.4644 ... -1.363 -1.363 -1.363
    REGION      (time) float64 -0.1738 1.435 -1.247 ... -0.1738 1.435 -1.247
    PPH_NUM     (time) float64 1.399 0.9215 -1.085 ... -0.3376 -0.1676 -1.085
    ...          ...
    TOR_F       (time) float64 2.231 0.9989 -0.8496 ... 0.9989 0.9989 -0.8496
    WINDSP_NUM  (time) float64 0.7173 0.4189 -1.372 ... 0.1801 0.7173 -1.372
    HAILSZ_NUM  (time) float64 1.051 0.493 -0.9944 ... -0.2507 -0.9944 -0.9944
    NEIGH_NUM   (time) float64 -0.1849 -0.133 -0.5377 ... -0.4907 -0.5355
    RMSE_NUM    (time) float64 0.4015 0.008813 -0.8782 ... -0.6374 -0.8182
    BS_NUM      (time) float64 0.2244 -0.04445 -0.7912 ... -0.6126 -0.7894

In [ ]:
# cluster with, k-means, knn, optics, birch, dbscan look here for use cases https://scikit-learn.org/stable/modules/clustering.html